## Read file

In [15]:
import pandas as pd
import numpy as np

In [16]:
file = open('articles.txt', 'r', encoding = 'utf8')
columns = ["pubId", "is_hourly", "seqId", "on_homepage", "canonicalUrl",
                   "firstScrape", "lang_iso", "lang_reliability", "title", "text"]


articles_dt = file.read().split('\n')[:-1]
pubId, is_hourly, seqId, on_homepage, canonicalUrl,firstScrape,lang_iso, lang_reliability, title,text = [],[],[],[],[], [],[],[],[],[]
for article in articles_dt:    
    row = article.split('\t')
    pubId.append(row[0])
    is_hourly.append(row[1])
    seqId.append(row[2])
    on_homepage.append(row[3])
    canonicalUrl.append(row[4])
    firstScrape.append(row[5])
    lang_iso.append(row[6])
    lang_reliability.append(row[7])
    title.append(row[8])
    text.append(row[9])
articles_df = pd.DataFrame()
articles_df['pubId'], articles_df['is_hourly'], articles_df['seqId'], articles_df['on_homepage'], articles_df['canonicalUrl'], articles_df['firstScrape'], articles_df['lang_iso'], articles_df['lang_reliability'],articles_df['title'], articles_df['text'] = pubId, is_hourly, seqId, on_homepage, canonicalUrl,firstScrape,lang_iso, lang_reliability, title,text
articles_df.shape

(213605, 10)

In [17]:
articles_df = articles_df[articles_df['lang_reliability'] == '1']
articles_df.shape

(211577, 10)

## Clean data

In [22]:
## text normzalization
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
import re

lemmatizer = WordNetLemmatizer()
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a paragrapy
        
        return: modified initial string
    """
    text = re.sub(r"[^\w\s]", '', text) ## Remove all non-word characters (everything except numbers and letters)
    text = text.lower() # lowercase text
    text_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in text_list]) # lemmatization
    return lemmatized_output

In [23]:
articles_df['text'] = articles_df['text'].apply(clean_text)

In [29]:
articles_df['title'] = articles_df['title'].apply(clean_text)

In [30]:
articles_df.head()

,pubId,is_hourly,seqId,on_homepage,canonicalUrl,firstScrape,lang_iso,lang_reliability,title,text
0,290,1,2,1,zerohedge.com/news/2018-07-19/i-told-you-so-tr...,7/19/2018 10:41:26 AM -04:00,en,1,i told you so trump hit out at eu after 5 bill...,by phoenix caput jul 19 2018 849 am the govern...
1,290,1,4,1,zerohedge.com/news/2018-07-19/trump-invites-pu...,7/19/2018 5:03:01 PM -04:00,en,1,trump invite putin to ahead of midterm zero hedge,by phoenix caput jul 19 2018 849 am the govern...
2,290,1,5,1,zerohedge.com/news/2018-07-19/gallup-shows-how...,7/19/2018 8:26:52 AM -04:00,en,1,gallup show how much american really care abou...,by knave dave jul 18 2018 111 pm this is exact...
3,290,1,7,1,zerohedge.com/news/2018-07-19/california-supre...,7/19/2018 8:26:52 AM -04:00,en,1,california supreme court block proposal to spl...,by knave dave jul 18 2018 111 pm this is exact...
4,290,1,9,1,zerohedge.com/news/2018-07-19/why-are-thousand...,7/19/2018 4:28:52 PM -04:00,en,1,why are thousand of tesla sitting in a field i...,by phoenix caput jul 19 2018 849 am the govern...


## bag of word

### 1. uni-gram

In [31]:
comb = []
for i in articles_df.index:
    comb.append(articles_df['title'][i] + ' ' + articles_df['text'][i])

In [32]:
comb[712]

' '

In [17]:
#title_text = pd.DataFrame(comb) 
#title_text.to_csv(r'title_text_df.csv')

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

cv = CountVectorizer()
text_counts= cv.fit_transform(comb)

In [34]:
text_counts.shape

(211577, 807149)

In [36]:
np.sum(text_counts[712,:].todense())
# empty context+title is 0 

0

In [37]:
## save result
np.save('BagOfWord_addlemmatization_output.npy', text_counts)  

In [34]:
# load saved result
#test = np.load('BagOfWord_output.npy', allow_pickle=True)
#np.asmatrix(test)[0,0]

### 2. bi-gram

In [38]:
cv2 = CountVectorizer(ngram_range=(2, 2))
text_counts2= cv2.fit_transform(comb)

In [39]:
text_counts2.shape

(211577, 9470591)

In [40]:
np.sum(text_counts2[712,:].todense())
# empty context+title is 0 

0

In [41]:
np.save('BagOfWord2_addlemmatization_output.npy', text_counts)  

## word2vec

In [42]:
from gensim.models import word2vec,KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [43]:
def word2vec(text): 
    article_vect = np.empty([len(text), 300])
    for i in range(0,len(text)):
        word_list = text[i].split(' ')
        for word in word_list:
            total_vec = np.zeros(300)
            if word in model:
                vec = model[word]
                total_vec = total_vec + vec
        article_vect[i,:] = total_vec/len(word_list)
    return article_vect

In [44]:
article_vect = word2vec(comb)

In [45]:
article_vect.shape

(211577, 300)

In [49]:
sum(article_vect[712,:])

0.0

In [56]:
np.save('word2vec_addlemmatization_output.npy', article_vect) 